In [41]:
import pandas as pd
import urllib.request

!apt install python3-tk ghostscript
!pip install camelot-py[cv]

from lxml import html
import requests

!pip3 install PyPDF2
!pip3 install re

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ghostscript is already the newest version (9.26~dfsg+0-0ubuntu0.18.04.12).
python3-tk is already the newest version (3.6.9-1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [96]:
# OBTÉM TODOS OS LINKS DISPINÍVEIS NA PÁGINA E EXTRAI O PRIMEIRO LINK QUE CONTÉM O PDF MAIS RECENTE

import camelot
from PyPDF2 import PdfFileReader
import re
import os

url = 'http://www.saude.ma.gov.br/page2.html'
page = requests.get(url)
webpage = html.fromstring(page.content)

links = webpage.xpath('//a/@href')
print(links)
nums = []

for link in links:
    if '.pdf' in link:
        splt = link.split('/')
        splt = splt[-1].split('nota')
        splt = splt[-1].split('.pdf')
        nums.append(int(splt[0]))
        
nums = list(set(nums))      
nums.sort(reverse=True)
print(nums)

links_pdfs = []

for i in range(0, len(nums)):
    links_pdfs.append(str('http://www.saude.ma.gov.br/downloads/notascovid/nota' + str(nums[i]) + '.pdf'))

print(links_pdfs)

achou_em = []

for i in range(0, len(links_pdfs)):

    url = links_pdfs[i]
    req = urllib.request.urlretrieve(url, r'covid19_MA_'+str(i)+'.pdf')

    # Open the pdf file
    object = PyPDF2.PdfFileReader(r'covid19_MA_'+str(i)+'.pdf')

    # Get number of pages
    NumPages = object.getNumPages()

    # Enter code here
    String = "DISTRIBUIÇÃO DE CASOS POR ORIGEM DE NOTIFICAÇÃO"

    # Extract text and do the search
    for j in range(0, NumPages):
        PageObj = object.getPage(j)
        Text = PageObj.extractText()
        if re.search(String, Text):
            # print(f"Pattern Found on Page: {j}, pdf: {links_pdfs[i]}")
            achou_em.append((r'covid19_MA_'+str(i)+'.pdf', j-1)) #j-1 coordenada de vetor diferente de coordenada de pdf (paginas)

print(achou_em)

['http://www.saude.ma.gov.br', 'page2.html', 'http://www.saude.ma.gov.br/saude', 'http://www.saude.ma.gov.br/downloads/notascovid/nota1.pdf', 'http://www.saude.ma.gov.br/downloads/notascovid/nota10.pdf', 'http://www.saude.ma.gov.br/downloads/notascovid/nota12.pdf', 'http://www.saude.ma.gov.br/downloads/notascovid/nota11.pdf', 'http://www.saude.ma.gov.br/downloads/notascovid/nota10.pdf', 'http://www.saude.ma.gov.br/downloads/notascovid/nota9.pdf', 'http://www.saude.ma.gov.br/downloads/notascovid/nota8.pdf', 'http://www.saude.ma.gov.br/downloads/notascovid/nota7.pdf', 'http://www.saude.ma.gov.br/downloads/notascovid/nota6.pdf', 'http://www.saude.ma.gov.br/downloads/notascovid/nota1.pdf', 'http://www.saude.ma.gov.br/downloads/notascovid/nota2.pdf', 'http://www.saude.ma.gov.br/downloads/notascovid/nota4.pdf', 'http://www.saude.ma.gov.br/']
[12, 11, 10, 9, 8, 7, 6, 4, 2, 1]
['http://www.saude.ma.gov.br/downloads/notascovid/nota12.pdf', 'http://www.saude.ma.gov.br/downloads/notascovid/nota11

[('covid19_MA_1.pdf', 0), ('covid19_MA_3.pdf', 0), ('covid19_MA_4.pdf', -1)]


In [118]:
import numpy as np

url = achou_em[0][0]
tables = camelot.read_pdf(url, pages = str(achou_em[0][1])) # editar o numero de páginas caso de erro
df = tables[0].df
df = df.T.reset_index(drop=True).T

df.to_csv('covid19_MA.csv')
df = pd.read_csv('covid19_MA.csv')
df = df.rename(columns={'0':'MUNICÍPIOS', '1':'SUSPEITOS', '2':'DESCARTADOS', '3':'CONFIRMADOS', '4':'TOTAL'})
df = df.iloc[2:df.shape[0]-1, 1:df.shape[1]]

cols = ['MUNICÍPIOS', 'SUSPEITOS', 'DESCARTADOS', 'CONFIRMADOS', 'TOTAL']
for col in cols:
    df[col] = df[col].map(lambda x: str(x).replace('-', '0'))

df['MUNICÍPIOS'] = df['MUNICÍPIOS'].map(lambda x: str(x).replace('\n', ''))
df['MUNICÍPIOS'] = df['MUNICÍPIOS'].map(lambda x: str(x).replace('.', ''))
df['MUNICÍPIOS'] = df['MUNICÍPIOS'].str.replace('\d+', '')
df = df.sort_values(['MUNICÍPIOS'], ascending=1)
df = df.reset_index(drop=True)
df.to_csv('covid19_MA.csv')
df

,MUNICÍPIOS,SUSPEITOS,DESCARTADOS,CONFIRMADOS,TOTAL
0,Chapadinha,1,0,0,1
1,Cidelândia,1,0,0,1
2,Coelho Neto,1,1,0,2
3,Davinópolis,2,0,0,2
4,Duque Bacelar,2,0,0,2
5,Estreito,1,1,0,2
6,Grajaú,1,0,0,1
7,Igarapé do Meio,1,0,0,1
8,Imperatriz,12,2,0,14
9,Jenipapo dos Vieiras,1,0,0,1
